In [1]:
import pandas as pd

from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor
import string
from bs4 import BeautifulSoup

# import boto3
import numpy as np

import json
import os
import time


ApplicationIntent="ReadOnly"
server="172.16.0.218"
database="Spatialrss"
username='RDUBSU6KMWWn3DQZNNzG_AI_Application'
password='>\Wt<C}H&[$222TEp`5krrZ```'
import pyodbc
conn = pyodbc.connect(
            'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password + ';ApplicationIntent=' + ApplicationIntent)
conn.setencoding(encoding="utf-8")
conn.setdecoding(pyodbc.SQL_CHAR, encoding="utf-8")
conn.setdecoding(pyodbc.SQL_WCHAR, encoding="utf-8")

In [2]:
def new_func(categoryname, brandid):
    q1 = f"""Select MAIN1.BrandID as Brandid, Ait.Response_Text as ResponseGenie,Main.Description as AgentReply
 From  {categoryname}.dbo.Tag_{categoryname} MAIN1 WITH(NOLOCK)
 Left Join {categoryname}.dbo.Tag_{categoryname} MAIN WITH(NOLOCK) on MAIN1.BrandID = MAIN.BrandID  
AND (Case when MAIN.ChannelGroupID in (7) Then Convert(Varchar(200),MAIN.PostID) ELSE Main.PostSocialID END)
    =(Case when MAIN1.ChannelGroupID in (7) Then MAIN1.ObjectID ELSE MAIN1.TweetIDorFbid  END) AND MAIN.IsBrandPost=1
LEFT OUTER JOIN {categoryname}.dbo.mstAISuggestedTokenDetails Ait WITH(Nolock) on  Ait.TagID=Main1.Tagid
Where MAIN1.IsBrandPost=0  AND MAIN1.ExistingTagID>0 
AND MAIN1.BrandID= {brandid}  
AND MAIN1.CreatedDate >='2023-09-24 18:30:00' AND MAIN1.CreatedDate <='2023-09-27 18:29:59'  """
    ans = pd.read_sql(q1, conn)
    return ans



In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')


def calculate_sentence_similarity(sentence1, sentence2):
    # Encode the sentences into embeddings
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

    return cosine_similarity.item()

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def operations(df):
    # drop duplicates
    df = df.drop_duplicates()
    # drop na from AgentReply
    df = df.dropna(subset=['AgentReply'])
    # drop na from ResponseGenie
    df = df.dropna(subset=['ResponseGenie'])
    # remove twitter handles
    df['ResponseGenie'] = df['ResponseGenie'].str.replace(r'@\w+', '', regex=True)
    df['AgentReply'] = df['AgentReply'].str.replace(r'@\w+', '', regex=True)
    # remove #tages
    df['ResponseGenie'] = df['ResponseGenie'].str.replace(r'#\w+', '', regex=True)
    df['AgentReply'] = df['AgentReply'].str.replace(r'#\w+', '', regex=True)
    # Remove punctuation
    df['ResponseGenie'] = df['ResponseGenie'].str.replace(f'[{string.punctuation}]', '', regex=True)
    df['AgentReply'] = df['AgentReply'].str.replace(f'[{string.punctuation}]', '', regex=True)
    return df

In [5]:
res = new_func("TataDigitalDb", 6954)

/tmp/ipykernel_366152/3258009590.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ans = pd.read_sql(q1, conn)


In [6]:
res

,Brandid,ResponseGenie,AgentReply
0,6954,"""@traumatizerr Hi, thank you for bringing this...","@traumatizerr Hi, please be assured that our t..."
1,6954,"""@traumatizerr Hi, thank you for bringing this...","@traumatizerr Hi, as discussed, we can confirm..."
2,6954,"""Thanks for providing additional details. We h...",None
3,6954,"""Hi Ajay Sharma, thank you for contacting Tata...",None
4,6954,"""Hi Ajay, thank you for sharing the requisite ...","@sharma999111 Hi Ajay, we have replied to you ..."
...,...,...,...
12549,6954,None,None
12550,6954,None,None
12551,6954,None,None
12552,6954,None,None


In [7]:
res = operations(res)

In [8]:
res

,Brandid,ResponseGenie,AgentReply
0,6954,Hi thank you for bringing this concern to our...,Hi please be assured that our team is activel...
1,6954,Hi thank you for bringing this concern to our...,Hi as discussed we can confirm that 100 offer...
4,6954,Hi Ajay thank you for sharing the requisite de...,Hi Ajay we have replied to you via DM Jerien
7,6954,Hi Ritesh we apologize for the inconvenience c...,Hi were disappointed to hear this We request y...
18,6954,Hi Kattar Hindu sorry to hear that you had a b...,Hi were disappointed to hear this We request y...
...,...,...,...
997,6954,Hi Navaz TECH thank you for your feedback We a...,Hi were disappointed to hear this We request y...
998,6954,Thank you for rating the app 5 stars How can I...,Hi were disappointed to hear this We request y...
999,6954,Thank you for giving us a perfect rating We re...,Hi were disappointed to hear this We request y...
1005,6954,Im sorry to hear that Can you please tell me h...,Hi were disappointed to hear this We request y...


In [9]:
score = []
for i, j in zip(res["ResponseGenie"].values, res["AgentReply"].values):
    score.append(calculate_sentence_similarity(str(i), str(j)))
res["score"] = score

In [10]:
res

,Brandid,ResponseGenie,AgentReply,score
0,6954,Hi thank you for bringing this concern to our...,Hi please be assured that our team is activel...,0.699237
1,6954,Hi thank you for bringing this concern to our...,Hi as discussed we can confirm that 100 offer...,0.128021
4,6954,Hi Ajay thank you for sharing the requisite de...,Hi Ajay we have replied to you via DM Jerien,0.479663
7,6954,Hi Ritesh we apologize for the inconvenience c...,Hi were disappointed to hear this We request y...,0.523158
18,6954,Hi Kattar Hindu sorry to hear that you had a b...,Hi were disappointed to hear this We request y...,0.340290
...,...,...,...,...
997,6954,Hi Navaz TECH thank you for your feedback We a...,Hi were disappointed to hear this We request y...,0.443824
998,6954,Thank you for rating the app 5 stars How can I...,Hi were disappointed to hear this We request y...,0.432028
999,6954,Thank you for giving us a perfect rating We re...,Hi were disappointed to hear this We request y...,0.457718
1005,6954,Im sorry to hear that Can you please tell me h...,Hi were disappointed to hear this We request y...,0.266694


In [13]:
average = res['score'].mean()
print(average)

0.46565347830197906


In [20]:
res.sort_values(by=['score'],ascending=False, inplace=True)

In [21]:
res

,Brandid,ResponseGenie,AgentReply,score
396,6954,Sorry for the frustration caused We would lik...,Hi Jasbir sorry for the frustration caused We...,0.984269
940,6954,We apologize for the inconvenience caused We u...,Hi Shahid we apologize for the inconvenience ...,0.972008
364,6954,Im sorry to hear that you are dissatisfied wit...,Hi Ashish we are sorry to hear that you are d...,0.954799
441,6954,Hi \n\nWe apologize for the inconvenience caus...,Hi Pragati we apologize for the inconvenience...,0.936780
769,6954,Hi we apologize for the inconvenience you hav...,Hi we apologize for the inconvenience you hav...,0.916853
...,...,...,...,...
600,6954,Thanks for rating us 5 stars We really appreci...,Hi were disappointed to hear this We request y...,0.144682
329,6954,Hi INDRANI BURRI thank you for your appreciati...,Hi were disappointed to hear this We request y...,0.138423
220,6954,\u0c39\u0c32\u0c4b \u0c2e\u0c40\u0c30\u0c41 \u...,Hi were disappointed to hear this We request y...,0.128419
1,6954,Hi thank you for bringing this concern to our...,Hi as discussed we can confirm that 100 offer...,0.128021
